# diffusers

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from diffusers import (
    DDPMPipeline,
    DDPMScheduler,
    UNet2DModel
)

## Load model

In [ ]:
# set model name
model_name = 'google/ddpm-cat-256'

In [ ]:
# load scheduler and model
scheduler = DDPMScheduler.from_pretrained(model_name)
unet = UNet2DModel.from_pretrained(model_name, use_safetensors=True)

print(f'Time steps:\n{scheduler.timesteps}')

In [ ]:
# load pipeline (u-net and scheduler)
ddpm = DDPMPipeline.from_pretrained(model_name)

## Generate images

In [ ]:
# reset time steps
scheduler.set_timesteps(num_inference_steps=50)

print(f'Time steps:\n{scheduler.timesteps}')

In [ ]:
# sample initial noise
sample_size = unet.config.sample_size

noise = torch.randn((1, 3, sample_size, sample_size))

In [ ]:
# run generative process
x_denoised = noise

# iterate over time steps
for t in scheduler.timesteps:

    # predict eps based on noisy x and t
    with torch.inference_mode():
        eps_pred = unet(x_denoised, t).sample

    # sample denoised x at previous step
    x_denoised = scheduler.step(eps_pred, t, x_denoised).prev_sample

In [ ]:
# show image
image_tensor = (x_denoised[0] / 2 + 0.5).clamp(0, 1)
image_tensor = image_tensor.permute(1, 2, 0)

fig, ax = plt.subplots(figsize=(4, 4))
ax.imshow(image_tensor.numpy())
ax.set_aspect('equal', adjustable='box')
fig.tight_layout()

## Generate with pipeline

In [ ]:
# run generative process
images = ddpm(num_inference_steps=25).images  # PIL images

In [ ]:
# show image
fig, ax = plt.subplots(figsize=(4, 4))
ax.imshow(np.asarray(images[0]))
ax.set_aspect('equal', adjustable='box')
fig.tight_layout()